In [ ]:
### Assessment Exercise for ASI

In [1]:
# 1. Download data
# MNIST
!wget http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
# CIFAR
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz

--2017-05-11 15:02:35--  http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 216.165.22.6
Connecting to yann.lecun.com (yann.lecun.com)|216.165.22.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9912422 (9.5M) [application/x-gzip]
Saving to: ‘train-images-idx3-ubyte.gz’

100%[======================================>] 9,912,422   2.96MB/s   in 3.2s   

2017-05-11 15:02:38 (2.96 MB/s) - ‘train-images-idx3-ubyte.gz’ saved [9912422/9912422]

--2017-05-11 15:02:38--  http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 216.165.22.6
Connecting to yann.lecun.com (yann.lecun.com)|216.165.22.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28881 (28K) [application/x-gzip]
Saving to: ‘train-labels-idx1-ubyte.gz’

100%[======================================>] 28,881      --.-K/s   in 0.1s    

2017-05-11 15:02:39 (263 KB/s) - ‘train-labels-id

In [5]:
!gzip -d train-images-idx3-ubyte.gz
!gzip -d train-labels-idx1-ubyte.gz
!gzip -d t10k-images-idx3-ubyte.gz
!gzip -d t10k-labels-idx1-ubyte.gz

!tar -xf cifar-10-python.tar.gz

gzip: train-labels-idx1-ubyte.gz: No such file or directory
gzip: t10k-images-idx3-ubyte.gz: No such file or directory
gzip: t10k-labels-idx1-ubyte.gz: No such file or directory


In [1]:
import struct
import numpy as np
# load data to note book
def read_mnist_data(images_file_name, labels_file_name):
#     images_file_name = "train-images-idx3-ubyte"
#     labels_file_name = "train-labels-idx1-ubyte"
    dataset = []
    with open(images_file_name, 'rb') as foi:
        with open(labels_file_name, 'rb') as fol:
            foi.read(4), fol.read(8) # omit magic number
            n_image = struct.unpack(">i", foi.read(4))[0]
            n_row = struct.unpack(">i", foi.read(4))[0]
            n_col = struct.unpack(">i", foi.read(4))[0]
            for i in np.arange(n_image):
                dataset.append((struct.unpack(">B", fol.read(1))[0],
                                np.array(struct.unpack(">" + "B"*(n_row*n_col), foi.read(n_row*n_col)))))
    return dataset
#             dataset.append()
#     print(magic, n_image, n_row, n_col)
mnist_training_data = read_mnist_data("train-images-idx3-ubyte", "train-labels-idx1-ubyte")
mnist_test_data = read_mnist_data("t10k-images-idx3-ubyte", "t10k-labels-idx1-ubyte")
print("Training data size: ", len(mnist_training_data))
print("Test data size", len(mnist_test_data))
print("Image size: ", mnist_training_data[0][1].shape)

('Training data size: ', 60000)
('Test data size', 10000)
('Image size: ', (784,))


In [2]:
# read CIFAR10
CIFAR_FOLDER = "./cifar-10-batches-py"
data_batch_file = ["data_batch_1", "data_batch_2", "data_batch_3", "data_batch_4", "data_batch_5"]
test_batch_file = "test_batch"

def unpickle(filename):
    import cPickle as pkl
    with open(filename, 'rb') as fo:
        dictionary = pkl.load(fo)
    result = [(dictionary["labels"][i], np.array(dictionary["data"][1]))
                     for i in np.arange(len(dictionary["data"]))]
    return result

cifar_full_data = []
for filename in data_batch_file:
    cifar_full_data.append(unpickle(CIFAR_FOLDER + "/" + filename))

    
cifar_test_data = unpickle(CIFAR_FOLDER + "/" + test_batch_file)
print(len(cifar_full_data))
print(cifar_test_data[0])

5
(3, array([235, 231, 232, ..., 178, 191, 199], dtype=uint8))


In [40]:
# 2/ 

In [12]:
# 3/ 1. Naive Bayes
import scipy

class NaiveBayesClassifier:
    def __init__(self, n_class, n_feature, feature_value_range):
        self.n_class = n_class
        self.n_feature = n_feature
        self.feature_value_range = feature_value_range
        self.probability_matrix = np.ones((n_class, n_feature, feature_value_range)) * 1.0
        self.probability_class = np.zeros((n_class)) * 1.0
        
    def train(self, training_data, training_label):
        # label already int from (0,.., n_class - 1)
        # training data (n_data x n_feature)
        for i in np.arange(len(training_data)):
            self.probability_class[training_label[i]] += 1.0
            for j in np.arange(len(training_data[0])):
                self.probability_matrix[training_label[i], j, training_data[i][j]] += 1.0
        self.probability_matrix = self.probability_matrix / np.sum(self.probability_matrix, axis=2).reshape((self.n_class, self.n_feature, 1))
        self.probability_class = self.probability_class / len(training_data)
        return self.probability_matrix
    
    def predict_one(self, test_data):
        result = 1.0
        for i in np.arange(len(test_data)):
            result = result * self.probability_matrix[:, i, test_data[i]]
        result = result * self.probability_class
#         print np.argmin(result)
        return (np.argmax(result), result)
    
    def predict_all(self, test_datas):
        result_array = np.zeros((len(test_datas), self.n_class))
        result_idx = np.zeros((len(test_datas),1))
        for i in np.arange(len(test_datas)):
            class_idx, prob_array = self.predict_one(test_datas[i])
            result_array[i, :] = prob_array
            result_idx[i] = class_idx
        return (result_idx, result_array)
    
    def predict_score(self, test_datas, test_labels):
        result_idx, _ = self.predict_all(test_datas)
        return np.sum([result_idx[i] == test_labels[i] for i in np.arange(len(test_datas))]) / float(len(test_datas))

In [13]:
# MNIST
classifier = NaiveBayesClassifier(10, 28*28, 2)
classifier.train([1.0*(x[1]>0) for x in mnist_training_data], [x[0] for x in mnist_training_data]);
print(classifier.predict_score([1.0*(x[1]>0) for x in mnist_test_data], [x[0] for x in mnist_test_data]))

/home/parallels/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/parallels/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:26: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0.8413


In [5]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
nb = MultinomialNB()
y_pred = (nb.fit([x[1] for x in mnist_training_data], [x[0] for x in mnist_training_data])
          .predict([x[1] for x in mnist_test_data]))
print(np.sum(y_pred == [x[0] for x in mnist_test_data]) * 1.0 / len(mnist_test_data))

0.8365
